
# DuckDuckGo + LangGraph (Multi‑Agent) with MCP Tools

This notebook shows a **multi‑agent** search system built with **LangGraph**, where a **Supervisor** agent routes tasks to a **Searcher** agent and an **Analyst** agent. The Searcher talks to a **DuckDuckGo tool over MCP (Model Context Protocol)**.  
If MCP is unavailable in your environment, the notebook **falls back** to a direct `duckduckgo-search` call so you can still run the graph immediately.

**Highlights**
- ✅ **MCP**: A tiny MCP server exposes `duckduckgo.text_search` (no API key required).  
- ✅ **LangGraph**: Supervisor → (Searcher ↔ MCP tool) → Analyst.  
- ✅ **Optional** OpenAI summary if `OPENAI_API_KEY` is set; otherwise a clean rule-based summary.


In [ ]:

# If needed, install dependencies. It's safe to re-run.
%pip install -q duckduckgo-search langgraph langchain-core langchain-community openai

# MCP pieces (optional). If these fail to install, the notebook still works using the fallback.
# The official Python SDK is typically named `mcp`.
try:
    %pip install -q mcp
    MCP_OK = True
except Exception as e:
    print("MCP install failed:", e)
    MCP_OK = False

print("MCP_OK:", 'MCP_OK' in globals() and MCP_OK)


In [ ]:

from __future__ import annotations

import os, sys, json, time, subprocess, atexit, asyncio, textwrap, uuid
from typing import TypedDict, List, Dict, Optional, Literal
from dataclasses import dataclass
from datetime import datetime

# LangGraph
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# DuckDuckGo direct (fallback)
from duckduckgo_search import DDGS

# Optional OpenAI summarize
OPENAI_AVAILABLE = False
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True if os.getenv("OPENAI_API_KEY") else False
    if OPENAI_AVAILABLE:
        openai_client = OpenAI()
except Exception:
    OPENAI_AVAILABLE = False

def now_iso():
    return datetime.now().isoformat(timespec="seconds")

print("OpenAI available:", OPENAI_AVAILABLE)



## MCP Tool Server (DuckDuckGo)

We write a tiny MCP server to disk that exposes one tool:
- `duckduckgo.text_search(query: string, max_results: number=8)`

We'll launch it as a subprocess and connect via stdio.


In [ ]:

mcp_server_path = "/mnt/data/mcp_duckduckgo_server.py"
server_code = r'''
import asyncio
import sys
import json
from typing import Any, Dict, List, Optional
from duckduckgo_search import DDGS

# Minimal MCP stdio loop using JSON RPC-like envelopes.
# This is intentionally lightweight for notebook demos.
# It supports: initialize, list_tools, call_tool

TOOLS = {
    "duckduckgo.text_search": {
        "name": "duckduckgo.text_search",
        "description": "Search DuckDuckGo and return top results",
        "input_schema": {
            "type": "object",
            "properties": {
                "query": {"type": "string"},
                "max_results": {"type": "integer", "minimum": 1, "maximum": 25}
            },
            "required": ["query"]
        }
    }
}

def ddg_search(query: str, max_results: int = 8):
    items = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results, safesearch="moderate", region="wt-wt"):
            items.append({
                "title": r.get("title"),
                "link": r.get("href"),
                "snippet": r.get("body"),
            })
    return items

async def ainput():
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(None, sys.stdin.readline)

async def main():
    # Send a basic "ready" notice
    print(json.dumps({"type":"ready"}), flush=True)
    while True:
        line = await ainput()
        if not line:
            break
        line = line.strip()
        if not line:
            continue
        try:
            msg = json.loads(line)
        except Exception as e:
            print(json.dumps({"type":"error","error":str(e)}), flush=True)
            continue

        mtype = msg.get("type")
        if mtype == "initialize":
            # acknowledge
            print(json.dumps({"type":"initialized"}), flush=True)
        elif mtype == "list_tools":
            print(json.dumps({"type":"tools","tools": list(TOOLS.values())}), flush=True)
        elif mtype == "call_tool":
            name = msg.get("name")
            args = msg.get("arguments") or {}
            if name == "duckduckgo.text_search":
                query = args.get("query","").strip()
                maxr = int(args.get("max_results", 8))
                results = ddg_search(query, maxr)
                print(json.dumps({"type":"tool_result","id": msg.get("id"), "result": results}), flush=True)
            else:
                print(json.dumps({"type":"tool_result","id": msg.get("id"), "error": "unknown tool"}), flush=True)
        elif mtype == "shutdown":
            print(json.dumps({"type":"bye"}), flush=True)
            break
        else:
            print(json.dumps({"type":"error","error":"unknown message type"}), flush=True)

if __name__ == "__main__":
    asyncio.run(main())
'''
with open(mcp_server_path, "w", encoding="utf-8") as f:
    f.write(server_code)

mcp_server_path



## Lightweight MCP Client (stdio)

For demo purposes, we implement a very small stdio client that speaks to the server script above using simple JSON messages.


In [ ]:

class SimpleMCPClient:
    def __init__(self, cmd: List[str]):
        self.proc = subprocess.Popen(
            cmd,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1
        )
        atexit.register(self.close)
        # wait for ready
        ready_line = self.proc.stdout.readline().strip()
        # print any early stderr noise
        time.sleep(0.1)
        if self.proc.poll() is not None:
            err = self.proc.stderr.read()
            raise RuntimeError(f"MCP server failed to start: {err}")
        # initialize
        self._send({"type": "initialize"})
        _ = self._recv()
    
    def _send(self, obj: Dict):
        if self.proc.stdin:
            self.proc.stdin.write(json.dumps(obj) + "\n")
            self.proc.stdin.flush()
    
    def _recv(self) -> Dict:
        if self.proc.stdout:
            line = self.proc.stdout.readline().strip()
            if not line:
                return {}
            try:
                return json.loads(line)
            except Exception:
                return {"type": "parse_error", "raw": line}
        return {}
    
    def list_tools(self) -> List[Dict]:
        self._send({"type": "list_tools"})
        msg = self._recv()
        return msg.get("tools", []) if msg.get("type") == "tools" else []
    
    def call_tool(self, name: str, arguments: Dict) -> Dict:
        msg_id = str(uuid.uuid4())
        self._send({"type":"call_tool", "id": msg_id, "name": name, "arguments": arguments})
        while True:
            msg = self._recv()
            if msg.get("type") == "tool_result" and msg.get("id") == msg_id:
                return msg
            if self.proc.poll() is not None:
                raise RuntimeError("MCP server exited unexpectedly")
    
    def close(self):
        try:
            if self.proc and self.proc.poll() is None:
                self._send({"type":"shutdown"})
                try:
                    self.proc.terminate()
                except Exception:
                    pass
        except Exception:
            pass


In [ ]:

def start_mcp():
    cmd = [sys.executable, "/mnt/data/mcp_duckduckgo_server.py"]
    client = SimpleMCPClient(cmd)
    return client

MCP_CLIENT = None
try:
    MCP_CLIENT = start_mcp()
    tools = MCP_CLIENT.list_tools()
    print("MCP tools:", [t["name"] for t in tools])
except Exception as e:
    MCP_CLIENT = None
    print("MCP client fallback:", e)



## LangGraph Multi‑Agent Design

Agents:
- **Supervisor**: reads the user query and routing hints, decides next step (`searcher` → `analyst` or finish).
- **Searcher**: calls the **MCP DuckDuckGo tool** (or direct fallback) to fetch hits.
- **Analyst**: summarizes results (rule-based) and optionally adds an OpenAI bullet summary.

The state flows Supervisor → Searcher → Analyst → END.


In [ ]:

class AgentState(TypedDict, total=False):
    query: str
    route: Literal["searcher","analyst","done"]
    results: List[Dict]
    answer_markdown: str

def supervisor_node(state: AgentState) -> AgentState:
    # simple policy: if no results yet, go search; else go analyze; else done.
    if not state.get("results"):
        return {"route": "searcher"}
    if not state.get("answer_markdown"):
        return {"route": "analyst"}
    return {"route": "done"}

def mcp_duckduckgo_search(query: str, max_results: int = 8) -> List[Dict]:
    # Prefer MCP if available
    if MCP_CLIENT is not None:
        res = MCP_CLIENT.call_tool("duckduckgo.text_search", {"query": query, "max_results": max_results})
        if "result" in res:
            return res["result"]
    # Fallback: direct search
    items = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results, safesearch="moderate", region="wt-wt"):
            items.append({
                "title": r.get("title"),
                "link": r.get("href"),
                "snippet": r.get("body"),
            })
    return items

def searcher_node(state: AgentState) -> AgentState:
    q = state.get("query","").strip()
    if not q:
        raise ValueError("Empty query for searcher")
    hits = mcp_duckduckgo_search(q, max_results=8)
    return {"results": hits}

def _markdown_from_results(results: List[Dict], query: str) -> str:
    if not results:
        return f"**No results** for `{query}`"
    lines = [f"### DuckDuckGo results for `{query}` (_{now_iso()}_)"]
    for i, r in enumerate(results, 1):
        title = r.get("title") or "Untitled"
        link = r.get("link") or ""
        snippet = r.get("snippet") or ""
        lines.append(f"{i}. [{title}]({link})\n   - {snippet}")
    return "\n".join(lines)

def _openai_summary(results: List[Dict], query: str) -> Optional[str]:
    if not OPENAI_AVAILABLE:
        return None
    bullets = "\n".join([f"- {r.get('title')} — {r.get('snippet')}" for r in results[:8]])
    prompt = f'''You are a helpful research assistant.
Summarize the most relevant findings for the query: "{query}".
Use 5-8 concise bullet points and include notable facts/dates if present.

Findings:
{bullets}
'''
    try:
        resp = openai_client.responses.create(model="gpt-4o-mini", input=prompt)
        parts = []
        for out in resp.output:
            if out.type == "message":
                for c in out.message.content:
                    if c.type == "text":
                        parts.append(c.text)
        txt = "\n".join(parts).strip()
        if txt:
            return "### Summary\n" + txt
    except Exception as e:
        print("OpenAI summarization failed:", e)
        return None
    return None

def analyst_node(state: AgentState) -> AgentState:
    results = state.get("results", [])
    query = state.get("query","")
    md = _markdown_from_results(results, query)
    add = _openai_summary(results, query)
    if add:
        md = add + "\n\n" + md
    return {"answer_markdown": md}


In [ ]:

graph = StateGraph(AgentState)
graph.add_node("supervisor", supervisor_node)
graph.add_node("searcher", searcher_node)
graph.add_node("analyst", analyst_node)

graph.set_entry_point("supervisor")

# conditional routing out of supervisor
def route_from_supervisor(state: AgentState):
    return state.get("route","done")

graph.add_conditional_edges("supervisor", route_from_supervisor, {
    "searcher": "searcher",
    "analyst": "analyst",
    "done": END
})

# linear edges
graph.add_edge("searcher", "analyst")
graph.add_edge("analyst", END)

memory = MemorySaver()
app = graph.compile(checkpointer=memory)

print("Graph ready.")


In [ ]:

def run_query(query: str) -> AgentState:
    init: AgentState = {"query": query}
    # supervisor -> (searcher -> analyst) -> end
    s1 = app.invoke(init)
    return s1

example = "latest LangGraph tutorials and docs"
final = run_query(example)
print(final.get("answer_markdown","")[:1200])


In [ ]:

# Try multiple queries; press Enter on an empty line to stop.
try:
    while True:
        q = input("\nQuery (blank to stop): ").strip()
        if not q:
            break
        result = run_query(q)
        print("\n" + result.get("answer_markdown",""))
except KeyboardInterrupt:
    pass
